# Instruction: Data Preprocessing and Visualization

# Basic Setup

In [ ]:
from IPython.display import display

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

# Data Preprocessing

In previous weeks, we have already seen several data preprocessing methods. Recall, for example, the instruction  on data exploration, basic visualization, and decison trees or the instruction session on regression. We learned preprocessing steps such as

* One-hot encoding
* Outlier detection
* Handling missing values
* etc.

Today we will see a few more techniques. However, note that there are many possiblities to achieve your preprocessing goals in Python and also this instruction just shows a subset of those.

## Missing values

To illustrate the handling of missing values we will use a running example based on the *diabetes data set*. The data set has the following attributes:

| Column | Description |
| ------ | ----------- |
|Pregnant | Number of times pregnant |
|PGC | Plasma glucose concentration a 2 hours in an oral glucose tolerance test |
|BP | Diastolic blood pressure (mm Hg) |
|Triceps | Triceps skinfold thickness (mm) |
|Insulin | 2-Hour serum insulin (mu U/ml) |
|BMI | Body mass index (weight in kg/(height in m)^2) |
|Pedi | Diabetes pedigree function |
|Age | Age (years) |
|Diabetes | Diabetes (yes/no) |

Let's first import the data set, have a look at the first couple rows, and check if `pandas` already infered reasonable data types.

In [ ]:
df = pd.read_csv('pima-indians-diabetes.csv')

In [ ]:
display(df)
display(df.dtypes)

In [ ]:
display(df.describe())
print(f'Total number of NAN values: {df.isna().sum().sum()}')

So far, the datatypes seem to be reasonable.

Next, we use the `describe()` method to print summary statistics on the data set to get a rough overview over the data.
Moreover, we check for existing NaN entries.

This summary can already help us to identify missing values (or the presence of potentially incorrectly recorded data, e.g., unrealistic min/max values). Recall from the lecture, not all missing values will be denoted by NaN. In fact, there are no NaN entries in this dataset. From the summary, we can see that several attributes have a minimum value of 0. From this, combined with domain knowledge, we can conclude that for several attributes the '0' value represents a missing/incorrect value.
These attributes are:
1. Plasma glucose concentration (PGC)
2. Diastolic blood pressure (BP)
3. Triceps skinfold thickness (Triceps)
4. 2-Hour serum insulin (Insulin)
5. Body mass index (BMI)

Before we can "blindly" fill these values, we investigate the missing values further.
To this end, we empirically investigate a few rows that are affected by missing values and count the number of missing values per dimension of the dataframe.
The latter helps us to identify attributes and rows that have suspiciously many NaN entries.


In [ ]:
df_tmp = df[['PGC', 'BP', 'Triceps', 'Insulin', 'BMI']] == 0
display(df.loc[df_tmp.any(axis=1)])
print('Colums NaNs')
display(df_tmp.sum(axis=0))
print('Row NaNs')
display(df_tmp.sum(axis=1).value_counts())
print(f'Total number of rows: {len(df_tmp.index)}')

del df_tmp

This very basic statistics already yield some interesting insights.
First, there are no obviously suspicious NaN patterns within the individual data records.
For example, sometimes you may observe that given that a record has missing values, the probability that it has multiple missing values is quite high (i.e., colloquial speaking, if there is a problem somewhere, then there are probably more problems as well).

Second, some attributes seem to have severe problems having more than 30% missing values. This, generally, raises the question if it makes sense to consider this attribute at all. (And you should definitely consider this when interpreting your results)
However, in this task, for the sake of exercise, we decide to impute the missing values.


Before we impute the missing values, we transform our data such that the invalid 0 values are actually represented as NaN. This is convenient because NaN values are ignored by functions such as `sum` or `count`.

In [ ]:
#replace 0 values by NaN values
df[['PGC', 'BP', 'Triceps', 'Insulin', 'BMI']] = df[['PGC', 'BP', 'Triceps', 'Insulin', 'BMI']].replace(0, np.NaN)
#count the number of NaN values to see whether we replaced all required values
print(df.isna().sum())

### Remove data entries with missing values
The simplest strategie to handle missing values is simply deleting all records that contain missing values. Pandas provides a simple function to do so.

In [ ]:
# count the number of rows and columns of the original dataset
print(df.shape)
# drop rows with missing values
#df.dropna(inplace=True)
df_tmp = df.dropna()
# count the number of rows and columns left in the dataset
print(df_tmp.shape)

### Impute missing values
Simply removing all data entries that contain missing values reduces the data set significantly. This way, we might loose valuable information important when training models on the data set (e.g. regression models, decision trees etc.). Moreover, it makes our analysis prone to having a survivors bias. Therefore, using other methods might be beneficial: 

- Filling in a constant value obtained through domain knowledge
- A value from another randomly selected record
- Mean / median / mode value of the attribute
- A value estimated by another predictive model

We can use the function `fillna()` from the `pandas` package for simply filling our missing values. The first argument of the function specifies the value that should replace the missing values. In the example below, we use the mean of each column.


In [ ]:
# fill missing values with mean column values
df_tmp = df.fillna(df.mean(axis=0))
# count the number of NaN values in each column
print(df_tmp.isna().sum())
display(df_tmp)

Alternatively, we can also choose among different *Imputers* provided by the `sklearn.impute` package. The example below demonstrates its usage. 

**Note that there are many other imputation methods already implemented in sklearn.**

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.NaN, strategy = "mean")
transformed_values = imputer.fit_transform(df)

print(np.isnan(transformed_values).sum())

In [ ]:
transformed_values

A minor problem when leaving the code above as it is, is that our input was a `pandas.DataFrame` while the output is a `numpy.ndarray`. Depending on our next steps, it might be better to transform this into a `pandas.DataFrame` that is consistent in terms of the selected index (in our current case this is simply 0-n) and the columns. Let's create such a `pandas.DataFrame`.


In [ ]:
df_tmp = pd.DataFrame(transformed_values, columns=df.columns, index=df.index)
df_tmp

### Your turn!
Handling missing values should have an effect on the quality of our trained models. To show this, take the *class-grades dataset* and train two different linear regression models to predict the final grade. Compare their accuracy scores.

* Model 1 should be trained on a data set with the missing values deleted.
* Model 2 should be trained on a data set with the missing values replaced by the mean of the attribute values.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# solution
grades = pd.read_csv('class-grades.csv', sep=",")
print(grades.head())



## Scaling / Normalization / Standardization

The terms scaling, normalization, and standardization are often used interchangeably. Most often normalization aims to rescale values into a certain range (e.g. [0,1]) and standardization rescales data to have a mean of 0 and a standard deviation of 1. **Note, that the `sklearn.preprocessing` package has chosen the general term "scaling" for both, normalization and standardization operations.**

In general, aside from the name of a function, it is important to know how it transforms your data and whether that is what you are aiming to achieve.

We take a closer look at the theory behind some of the scaling approaches in the Pen & Paper part of this instruction. Here, we focus on how to apply scaling to data in Python.


### Scaling attributes to a range
The *MinMaxScaler* transforms each feature to a given range (default: [0,1]).

In [ ]:
from sklearn import preprocessing
data = np.array([[ 1., -1.,  2.], [ 2.,  0.,  0.],[ 0.,  1., -1.]])

#creating the scaler
min_max_scaler = preprocessing.MinMaxScaler() 
#fitting the scaler to the data
min_max_scaler.fit(data)
#printing some information on the fittet scaler
print('The minimum value of each feature: ')
print(min_max_scaler.data_min_)
print('The maximum value of each feature: ')
print(min_max_scaler.data_max_)
print('The current range of each feature: ')
print(min_max_scaler.data_range_)
print(' ')


#transforming the data to the defined new range
transformed_data = min_max_scaler.transform(data)
print('The rescaled data: ')
print(transformed_data)

#We can also transform data other than the ones used to fit the scaler
print(min_max_scaler.transform([[2,2,2]]))

If the scaling function is bijective, we can also easily revert the scaling by using the `inverse_transform` function.

In [ ]:
# Rescale
data_rescaled = min_max_scaler.inverse_transform(transformed_data)
# Check if the rescaled array is close (by a certain tolerance) to the original data
assert np.allclose(data, data_rescaled)

The *MaxAbsScaler* scales each attribute such that the maximum absolute value of each feature in the training set will be 1.0. It's functions are similar to the MinMaxScaler.

In [ ]:
data = np.array([[ 1., -1.,  2.], [ 2.,  0.,  0.],[ 0.,  1., -1.]])

#creating the scaler
max_abs_scaler = preprocessing.MaxAbsScaler() 
max_abs_scaler.fit(data)

#transforming the data to the defined new range
transformed_data = max_abs_scaler.transform(data)
print('The rescaled data: ')
print(transformed_data)

print('Other data transformed by the same rescaler: ')
#We can also transform data other than the ones used to fit the scaler
print(max_abs_scaler.transform([[2,2,2]]))

#### Scaling to have a mean of 0 and standard deviation of 1

For many algorithms, problems arise if a feature has a variance that is significantly larger than the variance of other features. It might dominate the objective function and make the estimator unable to "learn" properly. To solve this problem, we can transform our data using the `scale` function provided by the `sklearn.preprocessing` package.


In [ ]:
data = np.array([[ 1., -1.,  2.], [ 2.,  0.,  0.],[ 0.,  1., -1.]])
scaler = preprocessing.StandardScaler()
scaled_data = scaler.fit_transform(data)
print('The scaled data: ')
print(scaled_data)
print(scaled_data.mean(axis=0))
print(scaled_data.std(axis=0))

### Your turn!
Scaling data can have an effect on the quality of our trained models. To illustrate this, take the *wine dataset* provided by sklearn datasets and train three different logistic regression models predicting the target. Compare their accuracy scores.

* Model 1 should be trained on the original data set.
* Model 2 should be trained on a data set scaled using the MinMaxScaler with range [0,1].
* Model 3 should be trained on a data set scaled to the attributes having a mean of 0 and a standard deviation of 1.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn import preprocessing

In [ ]:
df_wine = datasets.load_wine()

## Discretization / Binning
Discretization provides a way to partition continuous features into discrete values. Various algorithms and approaches exist to achieve this. 

Here, we introduce the KBinsDiscretizer, which discretizes the features into bins according to several paramters. 

By default, each feature is split into 5 bins. This can be configured with *n_bins* parameter. The parameter can be set using an integer (used for all features) or an array (different values for each feature possible).

By default, the output is one-hot encoded. This can be configured with the *encode* parameter:

* Onehot: Encodes the results with one-hot encoding and returns a sparse matrix.
* Onehot-dense: Encodes the results with one-hot encoding and returns a dense array.
* Ordinal: returns the bin identifier encoded as an integer value.

By default, the data is split into bins with equal number of data points. This can be configured with the *strategy* parameter:

* Uniform: All bins in each feature have identical width.
* Quantile: All bins in each feature have the same number of points.
* kmeans: Values in each bin have the same nearest center of 1D k-means cluster.

Below you can find example code for applying the KBinsDiscretizer.

In [ ]:
data = np.array([[ -3., 5., 15 ],[  0., 6., 14 ],[  6., 3., 11 ]])
discretizer = preprocessing.KBinsDiscretizer(n_bins=[3,2,2], encode='ordinal', strategy = 'uniform')
discretizer.fit(data)
discretized_data = discretizer.transform(data)
print('The discretized data:')
print(discretized_data)

print('The edges of each bin per feature:')
#displaying the edges of each bin per feature
print(discretizer.bin_edges_[0])
print(discretizer.bin_edges_[1])
print(discretizer.bin_edges_[2])

print('The number of bins per feature:')
#displaying the number of bins per feature (if bins are too small they can be deleted with a warning)
print(discretizer.n_bins_[0])
print(discretizer.n_bins_[1])
print(discretizer.n_bins_[2])

### Your turn!
Apply the KBinsDiscretizer on the data given below. The number of bins for each feature should be equal to the range of that feature. Use the encoding *onehot-dense* and choose the strategy such that all bins contains the same number of data points.
Print the discretized data as well as the bin boundaries for each feature.

In [ ]:
data = np.array([[-2, 1, -4,   -1],
                    [-1, 2, -3, -0.5],
                    [ 0, 3, -2,  0.5],
                    [ 1, 4, -1,    2]])

## Integrating Data
Let's have a look at how we can combine `DataFrame`s that have a common index. 
`Pandas` makes that very easy providing the `df.join` function.
*(If you want to use this function in your post-ids life, i.e., not in the assignment, check the types of joins first - left join, right join, outer join)*

In [ ]:
# Lets create wome random data sampled from normal distributions.
df1 = pd.DataFrame(np.random.normal(loc=0, scale=1, size=(100, 2)), columns=['x', 'y'], index=np.arange(100))
df2 = pd.DataFrame(np.random.normal(loc=2, scale=2, size=(100, 2)), columns=['x', 'y'], index=np.arange(100, 200))
df_data = pd.concat([df1, df2], axis=0)
df_classes = pd.DataFrame(np.concatenate([np.full((100, 1), 'c1'), np.full((100, 1), 'c2')], axis=0), columns=['label'], index=np.arange(200))

In [ ]:
# Join on the index (left-outer join)
df_merged = df_data.join(df_classes)

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
sns.scatterplot(x='x', y='y', hue='label', data=df_merged, ax=ax)
plt.show()

# Advanced visualizations
Recall basic visualization techniques from instruction session 2, such as box plots, bar charts etc. Here, we will work with more advanced techniques.

The following section shows an extract of the multitude of visualization possibilities. Python provides the advantage that many plots can be created with a few simple lines of code. Often, tuning the layout is more difficult than the visualization itself.

In the following, we will show some vizualizations as introduced in the lecture.

## Scatter Matrix

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

# we use the iris dataset
import seaborn as sns
data = sns.load_dataset('iris')

# Make the plot
scatter_matrix(data, alpha=0.2, figsize=(6, 6), diagonal='kde')
plt.show()

## Parallel Coordinate plot

In [ ]:
from pandas.plotting import parallel_coordinates
# we use the iris dataset again

# Make the plot
parallel_coordinates(data, 'species', colormap=plt.get_cmap("Set2"))
plt.show()

In [ ]:
import plotly.express as px

data['species_id'] = data['species'].replace(data['species'].unique(), [1, 2, 3])
fig = px.parallel_coordinates(data, 
                              color="species_id",
                              labels={"species_id": "Species",
                                    "sepal_width": "Sepal Width", "sepal_length": "Sepal Length",
                                    "petal_width": "Petal Width", "petal_length": "Petal Length", },
                              # color_continuous_scale=px.colors.diverging.Tealrose,
                              # color_continuous_midpoint=2
                             )
fig.show()

## Streamgraph
To visualize a streamgraph in Python we can make use of stackplots from the *matplotlib* package. Stackplots are generated by plotting different datasets vertically on top of one another rather than overlapping with one another.

In [ ]:
# the values for our x-axis
x = [1, 2, 3, 4, 5]
# the values that will be stacked on top of each other
y1 = [1, 1, 2, 3, 5]
y2 = [0, 4, 2, 6, 8]
y3 = [1, 3, 5, 7, 9]

# the labels for y1, y2 and y3
labels = ["Fibonacci ", "Evens", "Odds"]

#stacking our values vertically
y = np.vstack([y1, y2, y3])

fig, ax = plt.subplots()
#modifying the axis
ax.stackplot(x, y1, y2, y3, labels=labels, baseline='wiggle')
ax.legend(loc='upper left')
plt.show()

## Heatmap
We use the heatmap function from the *seaborn package* to create a heatmap. Note, we have to aggregate/pivot our data into the correct shape before the heatmap can be created.

In [ ]:
# Load the example flights dataset
# The example flights dataset is in long-format.
flights_long = sns.load_dataset("flights")
#In long-format, each data row can be interpreted individually. In this example each row contains a value for *year*, *month* 
#and *passengers*.
print('Long-format:')
print(flights_long) 
print(' ')

# We have to convert the data to wide-format to be able to use it for the heatmap.
#In wide-form, the values correspond to a combination of the *row category* and the *column category*. 
#When transforming from long-format to wide-format, categorical data will be grouped. In this example, 
#the wide-format displays the number of passangers for all combinations of *month* and *year*.
# Since there is only one value per month and year, we do not have aggregate our data in this example.
print('Wide-format:')
flights_wide = flights_long.pivot("month", "year", "passengers")
print(flights_wide)

# Draw a heatmap with the numeric values in each cell
fig, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(flights_wide, annot=True, fmt="d", linewidths=.5, ax=ax)
# For longer ticks it is sometimes helpful to rotate them
ax.tick_params(axis='x', labelrotation=60)

# Your turn!

Read the following json data about global temperature monthly variance and plot it on a heatmap, where
- the x-axis is the year 
- the y-axis is the month
- cells are the variance

You can use the seaborn function introduced above or the imshow function from plotly.express to make the heatmap more interactive.

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/freeCodeCamp/ProjectReferenceData/master/global-temperature.json')
print('baseTemperature:', df.baseTemperature[0])
df = pd.DataFrame(df['monthlyVariance'].tolist())
df

## Sankey
Even though `matplotlib` already has the functionality to create Sankey diagrams, the Sankey charts produced by `plotly` are admittedly much prettier.
Creating a basic Sankey chart using `plotly` is straightforward although it requires quite some boilerplate code.
In essence, you have to specify flows by means of and edge list and a list of flow values. Besides, you have to provide a list of vertex labels (label at index i is the label for vertex i). Finally, you can also provide lists of colors for the vertices and links. 


Consider the following artificial dataset where we have collected data on students who participated in an exam. 
Each row comprises the student who attempted the exam; whether it is his first, second, or third attempt; and the achieved grade.
After aggregating the data, we obtain the following Sankey diagram.

Evaluate the following visualization.


In [ ]:
import plotly.graph_objects as go

In [ ]:
# Define Data edge list and corresponding values
l_source = [0, 0, 0, 1, 1, 1, 2, 2, 2]
l_target = [3, 4, 5, 3, 4, 5, 3, 4, 5]
l_labels = ['Attempt 1', 'Attempt 2', 'Attempt 3', 'Grade [1,3)', 'Grade [3,4)', 'Grade 5']
l_values = [200, 400, 400, 100, 200, 100, 25, 100, 20] 

# Define Sankes diagram
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = l_labels,
      color = "blue"
    ),
    link = dict(
      source = l_source,
      target = l_target,
      value = l_values
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()


# Your turn!
In real-life no one is going to tell you which visualization technique to use for which data! 
Pretend that you are a data scientist who was just provided with the datasets used in this instruction session. Visualize the data using the previously presented techniques and/or other suitable techniques provided by the matplotlib (https://matplotlib.org/gallery/index.html) and the seaborn packages (https://seaborn.pydata.org/examples/index.html). Decide by yourself which part of the data you want to visualize using which plot.